In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from lib.imports import *
from lib.video_processor import VideoProcessor
from lib.misc import *

/home/tobi/.local/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [4]:
interpolate_rate=5
eyes_int_rate=1
eye_model_size=256
input_source = 0 # "example_video/example_vid_original.mp4"
form="onnx"


eye_batch_rate=(interpolate_rate//(eyes_int_rate+1))+1
convert_pt_to_onnx(model_path=f'models/eyes_{eye_model_size}_auto.pt', imgsz=eye_model_size, batch=eye_batch_rate, form=form)

processor = VideoProcessor(eye_model_size=eye_model_size, form=form)
out = processor.start(video_path=input_source, interpolate_rate=interpolate_rate, eyes_int_rate=eyes_int_rate, eye_batch_rate=eye_batch_rate, return_instead=False, disable_stats=False);

Ultralytics YOLOv8.2.9 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU (11th Gen Intel Core(TM) i7-1165G7 2.80GHz)
WARNING ⚠️ half=True only compatible with GPU export, i.e. use device=0
YOLOv8n-pose summary (fused): 187 layers, 3154073 parameters, 0 gradients, 8.6 GFLOPs

PyTorch: starting from 'models/eyes_256_auto.pt' with input shape (3, 3, 256, 256) BCHW and output shape(s) (3, 35, 1344) (6.2 MB)

ONNX: starting export with onnx 1.14.1 opset 17...
ONNX: simplifying with onnxsim 0.4.36...
ONNX: export success ✅ 0.5s, saved as 'models/eyes_256_auto.onnx' (12.1 MB)

Export complete (1.8s)
Results saved to /home/tobi/Desktop/eye_tracking/models
Predict:         yolo predict task=pose model=models/eyes_256_auto.onnx imgsz=256  
Validate:        yolo val task=pose model=models/eyes_256_auto.onnx imgsz=256 data=data/yolo_pose_dataset.yaml  
Visualize:       https://netron.app
Loading models/yolov8n-face-lindevs.onnx for ONNX Runtime inference...
Loading models/eyes_256_auto.onnx for ONNX Runtime 

In [ ]:
# write_video(out, "example_video/flexpressions_cropped.mp4")

## Training

In [ ]:
prepare_video_annotations_for_training(val_ratio=0.1, multiply_train_data=16)

In [ ]:
# if you don't want to track training with comet don't run this cell and set usecomet=False in the next cell
init_comet(api_key="your_api_key")

In [ ]:
for image_size in [64, 128, 256, 512]: # might get stuck before starting 2nd batch, if this happens just restart this cell
    train_model(pretrained_model='models/yolov8n-pose.pt', save_name=f"training_{image_size}_autotrain", imgsz=image_size, epochs=50, batch=64, workspace="comet_workspacename", usecomet=False)